# Исследование надежности заемщиков.

**Описание проекта**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - ответить на вопросы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о поведении пользователей мы получаем из файла `data.csv`. О качестве данных ничего не известно. Поэтому перед ответом на вопросы понадобится обзор данных.

Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Ответ на вопросы.

## Обзор данных

In [9]:
import pandas as pd
data = pd.read_csv('data.csv')
display(data.head(10))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

Для начала посчитаем сколько в таблице пропущенных значений.

In [2]:
display(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы нашли пропуски в двух столбцах: `days_employed` и `total_income`.

Пропуски в этих столбцах могут помешать исследованию и ответам на вопросы проекта. 

Найдем какую долю составляют пропущенные значения в каждом из столбцов с пропусками.

In [10]:
print(data.isna().mean())

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64


Пропущенные значения составляют около 10 процентов от общего количества.

Возможные пропуски в данных являлись технической или человеческой ошибкой при внесении данных.


In [4]:
print(data['total_income'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
display(data.isna().sum())

145017.93753253992


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Заполнили пропуски в столбце `total_income` медианным значением. Среднее значение может быть некорректно в данной ситуации из за очень большой разницы зарплат между людьми.

### Шаг 2.2 Проверка данных на аномалии и исправления.

В столбце `days_employed` 3 аномалии: 
1. Пустые значения
2. Сверхбольшие значения
3. Отрицательные значения

In [5]:
data['days_employed'] = abs(data['days_employed']) #изменяем все значения в столбце на положительные
for value in data['days_employed']: # изменяем все сверхбольшие значения, предполагая, что они записаны в часах 
    if value / 365 > 75:
        value = value / 24
med_days = data['days_employed'].median() 
data['days_employed'] = data['days_employed'].fillna(med_days)# Заменяем пустые значения на медианные
display(data.isna().sum())  

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Для сверхбольших значений взяли условное количество отработанных лет в виде 75, чтобы убрать данную аномалию и перевести ее в количество отработанных дней из отработанных часов. Заменили пустые значения на медианные, но нулевые значения трогать не будем, так как причина их появления неизвестна,а количество таких строк крайне мало.

In [15]:
data['children'] = abs(data['children'])
med_ch = int(data['children'].median())
data.loc[data['children'] == 20, 'children'] = med_ch
print(data['children'].unique())
print(data['children'].value_counts())

[1 0 3 2 4 5]
0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


Исправили аномалии в столбце `children`. Убрали отрицательное значение в виде -1 и заменили значение 20 на медианное, так как непонятно откуда оно могло взяться. Возможно из за человеческого фактора кто то случайно добавил либо 0 в конце, либо 2 в начале.

In [17]:
print(data['dob_years'].unique())#Встречаются нулевые значения
med = int(data['dob_years'].median()) #Медиана 
data.loc[(data['dob_years'] == 0), 'dob_years'] = med
print(data['dob_years'].unique())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51 59 29 60 55 58 71
 22 73 66 69 19 72 70 74 75]


Заменили 0 на медианное значение.

### Шаг 2.3. Изменение типов данных.

In [ ]:
data['total_income'] = data['total_income'].astype('int')
data.info()

### Шаг 2.4. Удаление дубликатов.

In [ ]:
print(data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True) # убрали строки-дубликаты
print(data.duplicated().sum())

Убрали строки-дубликаты.
Теперь займемся неявными дубликатами.

In [ ]:
print(data['education'].unique())

Приведем все к нижнему регистру, тем самым убрав неявные дубликаты из столбца `education`.

In [ ]:
data['education'] = data['education'].str.lower()
print(data['education'].unique())
print(data['family_status'].unique()) #нет неявных дубликатов
print(data['gender'].unique())#нет неявных дубликатов
print(data['income_type'].unique())#нет неявных дубликатов
print(data['purpose'].unique())#нет неявных дубликатов

Проверили остальные столбцы на неявные дубликаты. Их нигде не оказалось, разве что в столбце `gender` есть XNA, но я думаю что это пропуск и встречается он только в одной строчке.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новый датафрейма `education_dict` и `family_status_dict`, каждому уникальному значению из `education` соответствует уникальное значение `education_id` — в первом, тоже самое с `family_status`.

Также удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`.

In [ ]:
education_dict = data[['education','education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop = True)
display(education_dict)#Проверка
family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop = True)
display(family_status_dict)#Проверка
data = data.drop(['education', 'family_status'], axis=1)
display(data.head(10))#Проверка

### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создаем столбец `total_income_category` с категориями:
1. 0–30000 — `E`;
2. 30001–50000 — `D`;
3. 50001–200000 — `C`;
4. 200001–1000000 — `B`;
5. 1000001 и выше — `A`.

In [ ]:
def income_category(total_income):
    if total_income >= 0 and total_income<= 30000:
        return 'E'
    if total_income >= 30001 and total_income<= 50000:
        return 'D'
    if total_income >= 50001 and total_income<= 200000:
        return 'C'
    if total_income >= 200001 and total_income<= 1000000:
        return 'B'
    if total_income >= 1000001:
        return 'A'
data['total_income_category'] = data['total_income'].apply(income_category)    
display(data)

### Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:
1. `операции с автомобилем`,
2. `операции с недвижимостью`,
3. `проведение свадьбы`,
4. `получение образования`.

In [ ]:
data['purpose'].unique()
def purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жи' in purpose:
        return 'операции с недвижимостью'
    if 'сва' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'
data['purpose_category'] = data['purpose'].apply(purpose_category)    
display(data)

### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

Зависимость оказалось неявной, так как в семьях с 5 детьми 100% выплачиваемость кредитов, но для выборки в 9 таких клиентов это очень мало. Если бы появилась семья с невыплаченным в срок кредитом, то ratio было бы уже 10%. Также можем увидеть, что семьи с 3 детьми имеют больший процент вылпачиваемости кредитов в срок в отличие от семей с 1, 2 и 4 детьми. Поэтому данная зависимость является слабой.

In [ ]:
children_result = pd.DataFrame()
children_result['sum'] = data.groupby('children')['debt'].sum()
children_result['count'] = data.groupby('children')['children'].count()
children_result['ratio'] = children_result['sum']/children_result['count']
display(children_result.sort_values('ratio'))

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

Зависимость присутствует. Овдовевшие люди и разведенные намного чаще платят кредиты в срок, нежели люди, которые никогда не были в браке

In [ ]:
family_status_result = pd.DataFrame()
family_status_result['sum'] = data.merge(family_status_dict, on = 'family_status_id', how = 'left').groupby('family_status')['debt'].sum()
family_status_result['count'] = data.merge(family_status_dict, on = 'family_status_id', how = 'left').groupby('family_status')['family_status'].count()
family_status_result['ratio'] = family_status_result['sum']/family_status_result['count']
display(family_status_result.sort_values('ratio'))

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Здесь зависимость не наблюдается, так как выборки людей с доходами меньше 30000 и больше 1000000 крайне малы и по ним мы не можем делать какие-то выводы. По таблице видно, что люди с почти самым маленьким заработком выплачивают в срок намного больше, чем люди с почти самым большим заработком.

In [ ]:
income_total_result = pd.DataFrame()
income_total_result['sum'] = data.groupby('total_income_category')['debt'].sum()
income_total_result['count'] = data.groupby('total_income_category')['total_income_category'].count()
income_total_result['ratio'] = income_total_result['sum'] / income_total_result['count']
display(income_total_result.sort_values('ratio'))

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

Зависимость присутствует. Люди берущие кредит для операции с недвижимостью чаще всего отдают кредит в срок, в то время как кредиты, выданные для покупки автомобиля и для получения образования, отдаются хуже всего. Кредит для проведения свадьбы отдается также часто как и кредит на недвижимость.

In [ ]:
purpose_result = pd.DataFrame()
purpose_result['sum'] = data.groupby('purpose_category')['debt'].sum()
purpose_result['count'] = data.groupby('purpose_category')['purpose_category'].count()
purpose_result['ratio'] = purpose_result['sum'] / purpose_result['count']
display(purpose_result.sort_values('ratio'))

## Общий вывод:

Был произведен обзор данных, затем их преобработка, при которой мы выявили аномалии, пропуски в данных, явные и неявные дубликаты. Произвели категоризацию данных по уровню дохода, семейному положению, образованию, а также по целям кредита.

Мы провели все 3 этапа исследования, о которых написали вначале. По данным которые получили можно сказать следующее:

1. Чем больше детей, тем реже выплачиваются кредиты. 
2. Разведенные и овдовевшие люди чаще выплачивают кредиты в срок, чем те кто не был еще в браке.
3. Кредиты, которые берут для недвижимости или проведения свадьбы чаще отдаются вовремя, чем кредиты для получения образования и покупки автомобиля.

**ИТОГИ**
1. Зависимость между количеством детей и возвратом кредита в срок `Имеется`
2. Зависимость между семейным положением и возвратом кредита в срок `Имеется`
3. Зависимость между уровнем дохода и возвратом кредита в срок `Не имеется`
4. Цели кредита влияют на его возврат в срок `Влияют`